Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import nltk
import lightgbm as lgb
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.head())
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


С помощью регулярных выражений обработаю текст и лемматизирую для дальнейшего исследования

In [3]:
nltk.download('wordnet')

m = WordNetLemmatizer()

def clear_lemmatize(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    lemm_list = nltk.word_tokenize(text)
        
    return ' '.join(m.lemmatize(i, wordnet.VERB) for i in lemm_list)

data['lemm_text'] = data['text'].apply(clear_lemmatize)
display(data.head())

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It..."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestions on ...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb..."


In [4]:
x = data['lemm_text']
y = data['toxic']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=12345)

Векторизация счетчика слов с учетом обработки стоп слов.

In [6]:
stopwords = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
X_train = count_tf_idf.fit_transform(X_train)

### Вывод 

Убраны лишние символы, лемматизированы комментарии. Текст преобразован в векторный вид. Данные разбиты на обучающую, тестовую и валидационную выборки.

# 2. Обучение

### LogisticRegression

In [7]:
model_LR = LogisticRegression(class_weight = 'balanced', random_state=12345)
model_LR.fit(X_train, y_train)
X_valid_LR = count_tf_idf.transform(X_valid)
pred_LR = model_LR.predict(X_valid_LR)
f1_score(y_valid, pred_LR)

0.7488559892328398

Немного не дотягивает до необходимой метрики, подберу параметры

In [8]:
%%time

pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    )
])

param_grid = [
    {
        'model' : [LogisticRegression(random_state=12345, solver='liblinear')],
        'model__penalty' : ['l1', 'l2'],
        'model__C' : list(range(1, 15, 3))
    }
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', 
                    cv=3, verbose=True, n_jobs=-1)
best_grid = grid.fit(X_train, y_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.9min finished


Best parameters is: {'model': LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7756536417303713
CPU times: user 2min 37s, sys: 1min 21s, total: 3min 58s
Wall time: 4min


In [9]:
model_LR_1 = LogisticRegression(random_state=12345, C=4, penalty='l1', solver='liblinear', max_iter=200)
model_LR_1.fit(X_train, y_train)
X_valid_LR_1 = count_tf_idf.transform(X_valid)
pred_LR_1 = model_LR_1.predict(X_valid_LR_1)
f1_score(y_valid, pred_LR_1)

0.7850467289719626

In [10]:
X_test_LR_1 = count_tf_idf.transform(X_test)
prediction = model_LR_1.predict(X_test_LR_1)
f1 = f1_score(y_test, prediction)
print('F1:', f1)

F1: 0.7943548387096774


### CatBoost 

In [11]:
%%time

cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=100, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=12345)
cat_model.fit(X_train, y_train, verbose=20)
X_valid_cat = count_tf_idf.transform(X_valid)
prediction = cat_model.predict(X_valid_cat)
f1 = f1_score(y_valid, prediction)
print('F1 CatBoost:', f1)

0:	learn: 0.4510978	total: 6.01s	remaining: 9m 55s
20:	learn: 0.7416700	total: 1m 44s	remaining: 6m 34s
40:	learn: 0.7765712	total: 3m 23s	remaining: 4m 53s
60:	learn: 0.7924512	total: 5m 2s	remaining: 3m 13s
80:	learn: 0.8050432	total: 6m 41s	remaining: 1m 34s
99:	learn: 0.8111116	total: 8m 16s	remaining: 0us
F1 CatBoost: 0.7557980900409276
CPU times: user 9min 51s, sys: 19.5 s, total: 10min 10s
Wall time: 10min 12s


Необходимая метрика достигнута

In [12]:
X_test_cat = count_tf_idf.transform(X_test)
prediction = cat_model.predict(X_test_cat)
f1 = f1_score(y_test, prediction)
print('F1 CatBoost:', f1)

F1 CatBoost: 0.7564410855376159


# 3. Выводы

Обе модели получили значение метрики F1 больше 0.75. Лучший показатель после подбора параметров у модели линейной регрессии.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны